In [1]:
from TDmodel import TDmodel

In [2]:
model = TDmodel('SARSA',8,0.95,1,False)

In [3]:
V_log, rewards, states, paths = model.train()

Training time: 13.994261026382446


<h2>Start training Visualization</h2>

In [15]:
start = 0
Vs=iter(V_log[start:])
R=iter(rewards[start:])
S=iter(states[start:])
P=iter(paths[start:])

In [17]:
import jupyterlab_dash
viewer = jupyterlab_dash.AppViewer()

import datetime

import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly
from dash.dependencies import Input, Output
import numpy as np

L1=[0,0,0,0,0,0]
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div(
    html.Div([
        html.H4('SARSA Value update'),
        html.Div(id='live-update-text'),
        dcc.Graph(id='live-update-graph'),
        dcc.Interval(
            id='interval-component',
            interval=1*200, # in milliseconds
            n_intervals=0
        )
    ])
)

@app.callback(Output('live-update-text', 'children'),
              [Input('interval-component', 'n_intervals')])
def update_metrics(n):
    style = {'padding': '5px', 'fontSize': '16px'}
    return [
        html.Span('Iter: {0:.2f}'.format(n_intervals), style=style),
    ]



@app.callback(Output('live-update-graph', 'figure'),
              [Input('interval-component', 'n_intervals')])
def update_graph_live(n):
    dim=8
    _V = next(Vs)
    V = [list(v) for v in list(_V.reshape(dim,dim))]
    L1.append(np.linalg.norm(_V))
    rewarded = next(R)

    s=next(S)
    X=int(s/dim)
    Y=(s%dim)
    xy_map = {0:0.05,1:0.15,2:0.30,3:0.44,4:0.55,5:0.70,6:0.80,7:0.90}
    Y=xy_map[Y]
    X=xy_map[X]

    p=next(P)
    state_to_xy = lambda state: (int(state/dim), state%dim)
    m=[*zip(*[*map(state_to_xy,p)])]
    #     X_path = list(map(lambda x: X_map[x],m[0]))
    #     Y_path = list(map(lambda y: Y_map[y],m[1]))
    X_path = m[1]
    Y_path = m[0]

    fig = plotly.tools.make_subplots(rows=1, cols=1, vertical_spacing=0.2)
    fig['layout']['margin'] = {
        'l': 30, 'r': 10, 'b': 30, 't': 10
    }
    fig['layout']['legend'] = {
        'x': 0, 'y': 1, 'xanchor': 'left'
    }
    
    if rewarded:
        img_source= "https://image.shutterstock.com/image-vector/trophy-cup-award-vector-icon-450w-592525184.jpg"
    else:
        img_source = "data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxESEhUQExEQFRIXFhcYFxcWFxgXGBoYFRUXFxYTHxgYHyggGRsoHRcVITUhJSkrMC8uFx8zODMtNygtLi0BCgoKDg0OGxAQGzAmICYtLS8wNS0tLS0tLy0tLS0tLSstLS0uLS0tNS0tLS0tLS8tLS0tLS0tLTUtLS0tLS0tLf/AABEIAOEA4QMBEQACEQEDEQH/xAAcAAEAAgMBAQEAAAAAAAAAAAAABgcCBAUBAwj/xAA7EAABAwEDCQUIAwACAgMAAAABAAIDEQQFYQYSITFBUWKB8BNSkaHhByIyQnFyktEUscEjM8LxQ4Ki/8QAGwEBAAIDAQEAAAAAAAAAAAAAAAEFAgQGAwf/xAAxEQACAgIBAwEGBQUAAwAAAAAAAQIDBBEFEiExQRNRYbHh8CIjMnGhBoGR0fEzQsH/2gAMAwEAAhEDEQA/ALuzeEKCdjN4Qg2M3hCDZpXvekFljMs7mMYN+snugbTgolJRW2etNM7pdMFtkBt/tUbWkFkLm96QhteQWrPMhEvqP6etmtykY2L2q6f+eyEN3xuDqciohmwkTd/TtsVuMie3JfFntcfawOY9uo72nukawVtxkpLaKG+iyiXTYtHQzeEKTx2M3hCDYzeEINjN4Qg2M3hCDYzeEINjN4Qg2M3hCDYzeEINjN4Qg2M3hCDYzeEINjN4Qg2M3hCDZrXlborPGZpnMZG3W5xp9BicEbSW2eldc7ZdMO7K/vH2qMqRZ7K547zyGg8ta1J5kIl7j/0/dNbk9HxsntVIP/NY/d3xuqRyKxjmwZ62/wBOWRW4yJ3cF/Wa2Mz4HNdT4mnQ5p3FuxbcZqS7FDkY1tEumxHUzeELI19jN4Qg2M3hCDYzeEINimB8fVSBTA+PqgObf99Q2OEzzEgDQBX3nO2NArpKxlJRW2e+Pjzvn0QKZvC2Wi9LR2jwQ0fAz5Y27/qqm66VsumJ3GJiVYNXVLz8zuWS4IWDS0vO8/pYxpijzsz7ZPs9L4Gc9xwOFOzzcW6FLpg/QxhnXRf6t/ucKM2i7JxPCSWnWPleO6aaipqslTLT8HrfRVn1PtqX33RcGTV/w22ETRE11PYT7zHbWkV89qtYTUltHFZWNPHn0T/6damB8fVZmsKYHx9UApgfH1QCmB8fVAKYHx9UApgfH1QCmB8fVAKYHx9UApgfH1QCmB8fVAKYHx9UApgfH1QGlfF6Q2WJ08zs1jRv0k7GgV0k7ljKSits9aaZ3TUIeSmr2vK0XrPUgthafcZXQ0d473FVd10rH0xO0wsKrBq6592/5+h17LcMDBpZnHef0sI0xRNmddJ9npfAytFxwOFOzzTvboR0wfoRDOui/O/3I66O0XfO2eIkHf8AK8bWOoorslTLT8G1bVVn1Na7+7/6W/knlNDbos9lRI3RJGTpafHSNxVtCamto4jMw7MafTLx6M7lMD4+q9DTFMD4+qAUwPj6oBX7vAoDm3/fcNjhdPK4gDUPmc7Y0DesZSUVtnvj4875qEEUzb7baL0tHaPqGj4W62xt34uVTddK2XTE7fExKsCrql5/lskkMUVmj15rRrO8/tSlGETWnOzIs97Ixed+ySO9xzmM2AaCcSVqTubfYuMfBhWvxLbPLvv6WM+8S9u0HXyKQukvJN+DXYvwrTJTHJFaI9HvMOsbQd2BW2nGaKWUbMez3NEca60XbaO3hJLDrGx7e6ccVFdkqZafg2b6Ks+rx+L77ouDJrKCG2wiaJzq6nsPxMdtaf3tVrCakto4vKxZ48+if/TrV+7wKzNYV+7wKAV+7wKAV+7wKAV+7wKAV+7wKAV+7wKAV+7wKAV+7wKAV+7wKA0r3vSGzROnmeWsaMak7GgbSVjKSits9aaZ2zUILuU1et52i9Zw4gthafcZrDR3jvcqy66Vj6YnaYWFVg1dc+7+f0O/DFFZ49ea0ayduKxSjBHlKdmRZ72yN3llC95pGSxu/af0tad7fgt8fj4QW593/BrWK+Zo3VL3OG0E1/8ASwjbKLPa3DqsWtafwJXFLFaY97TrG0H9rbTjZEpJRsxrPiRl7LRd84nhcQdh2PbtY5Y12Splp+Dctqqz6mmu/r/tFv5J5TQ26LPYS2RuiSM62n/RuKtoTU1tHE5mHPGn0y8ejO5X7vAr0NMV+7wKA+FutjIY3zPfRjGlzjo1AVKhvS2Z11ynJRiu7KOvC32i9LR2jqhgPuNPwsZvO9yqb7ZWS6YndYWJVg09UvPzZIo2RWWLXRo1naT/AKUSjXE8JSsybPj8iJXtejp3adDBqb/pxWnZY5su8bGjSviaC8jaCA3LtvB8Ls5p0bRsK9ITcH2PC/HjdHTJjBPFaY97TrB1g7lupxmignCzHs+JHmPtF2T9vAatOsfK9vddjildkqXp+DYvoqz6u6/F990XBk3lBDbYRNE6h+ZhpnMdtaf2rSE1JbRxeTizx59El9TrV4v6WRrCvF/SAV4v6QCvF/SAV4v6QCvF/SAV4v6QCvF/SA073vWGzROmlkzWNGFSdjQNpUOSS2z1ppnbNQgu5TF7XpaL2nqashafcZsaO8d7lWXXSsfTE7TCwqsGvrn3fz+h3o2RWaPWA0aztJ/0rFKMInnKVmRZ8SJ3verpztDBqb/pxWnZY5l5jYsaV8TnLyNoIDZu+3PhdnNP1Gwjcs4TcXtHjfTG6PTImME8Vqj3g62nWD1tW6nGyJQzhZjWEbeLRd84nhdq1HY9u1jgornKmWvQ27qqs+l7Xf1/2i68n74Za7PHaWGgeNINKtcNDmn6Gqt4y6ls4TIolRY65eh0a8X9KTwIb7WbS5t3uaD8b2NNNxNSPJeV71AtOHgpZK+BELnDLPZhISBUZx3ncFWQ1GHUzp8nquv6I+nZEavS8nzuqdDR8Ld3qtWyxzZb4+PGmOl5NJeZsBAEAQG1d9vfC7OafqNhwWcJuL2jyupjdHpkTKzWmO0x7CDoc06wetq3YuNiKCyuzGs+TOAx9puycTwOq06wfhe3uuG/FK7JUvT8HvfRVn1d1qX33RcOTeUEVthE0TgO8w/E121p0+atITUltHFZOLPHn0SX1OtXEdc1ma4riOuaAVxHXNAK4jrmgFcR1zQCuI65oDSve9IrNE6aV7WsaOZwArpKxlJJbZ600ytmoQXcpi9r0tN7T1PuQtPuM2NHeO9yrLrpWPpidph4VWDX1y7v5/Q7rBFZYtYDR4k/6Vj+GuJ5t2ZNnx+REb1vR87qnQ0am7sfqtOyxzZe42NGmOl595oLzNgIAgCA2LDbHxOz2HTtGwjcVlGTi9o87qo2x6ZEu7Zlqgdq1Go2tcAtzasgUXRPFvX3tHY9jNod2NoiroZKCP8A7DT/AEt/Ek3Apf6grUb00WNXEdc1tFARv2h3W60WCVjW1e0CRoGslmmnhVedseqLRvcdcqsiLb7eP8lMS3kZIo49jR4lUVra/D7j6FRVHqdi8s1V5G0EAQBAEAQGxYrY+J2ew6d2wjcVlGTi9o87aY2x6ZEzslqjtUZBA3OadY63rdjJWRKC2qzGs2v7M4LHWi7JxPCatJ0g/C9u1jscUrslS9Pwe19NWfW01qX33RcWTd/w22ETRU3OaT7zXbWnR5q0hNSW0cVk408efRM61DuHXJZmuKHcOuSAUO4dckAodw65IDSve84rNE6aUtaxo36SdgAppKxk0ltnrTVK2ahDyUxe152i9p84jMhafcZsaO8d7iqy66VkumJ2mHh1YNXXLu3/AD9DtDsrJFsAHi4/tY/hriYfmZVn3pEQvO8XzOznaBsbsHqtOc3Jl7j48aY6Xk01ge4QBAEAQBAbd33gYS47C0gjloK9K5NPSNfJqjZHb9O5aPsju10djMzm0M7y8V7o0NPPSVdY8OmBwvNXqzI0vQnNDuHXJbBTnlOFQCpfaFkQYS62WZhMRJdLGPlrre0bsFo5OMpLaOp4fl2mqrH+3xIG1wIqFUyi09M6+MlJbR6oMggCAIAgCA+1ktT43B7DQjwOBWUZOL2jzsrjZHpkiZWK2RWqMtLRWnvN3Yj9rcjJWIobarMae0/2ZxGOtF2TieE1ZXSD8L27WOG/FTXZKp6fg9bqas+ppr8X33RcWTd/Q22ETRfRzTSrTuKtISUltHFZONPHn0TOtThWRrinCgNO97yhs0TppSGsaMKk7ABtKiTUVtnrTVO2ahDyUve152i9bRUgsgafcZsaO8d7iqy652Ppidph4dWDV1T7t/z9Dskw2SLVo/8A04rD8NcTFKzKs+9Ih94298zs53IDUBuWpObk9svaKI0x1E1VgewQBAEAQBACVKTZDaS2yVZC5GutzhPKC2ytP0MhGwcOKs8bF9ZHM8vy/s/y6/PyLpjia0BrWANAAAFKADUFYnGuTb22ZU4UIFPu8SpB4Wg6CDTmhJUvtByHMBda7Kw9lrkjHy73t4cNi0MnGUltHU8Ry72qrX9SBtcDpCqnFp6Z10ZKS2j1QSEAQBAEAQH1s1odG4PYSHDqn0Uxk4vaMJwjOPTJdiY2C3RWqMscPep7zf8AQtyMo2LTKK6meNPqj49Gcdj7RdloE8JJYTQj5Xt2sdjisq7JVS0/B6XU1Z9TTX4vvui4sm7+htsIliJ3ObU1a7cVZwmpLaOLycaePPpmbV73nDZonTSuLWNxOk7AN5Uykkts86ap2zUILuUve952i9bRU5zYG/AzY0d473FVt1zsl0xO0w8OrCq6p+X/AD9DrSSQ2OKgBwG1x3rBuNcSIxsyrN/aIhbra+Zxe8/QbANwWnKTk9svKaY1R6YmssT1CAIAgCAIASiWyG0ltkryEyNdbXCaYObZWn6GQjYOHFWeNjf+0jmuX5f2f5dfn5FzwQsY0MY3Na0UAGgADUFY60cdKTk9tn0p93iVJiKfd4lAK4nw9EArifD0QHhodBrT6eiAqX2hZEdgXWuytJiJrJGB8O97R3cNi0cnGUltHVcRy72qrX+3x+pAmurpCqXFp6Z1sZKS2j1QZBAEAQBAEB9IJnMcHtJDhqIUptPaMJxU10y8Ewu68I7UwxvpnU0t38QW5CasWmUd9E8afXDx7zkxvtF2WgTwuOYdBHyvbtY4b8VlXY6pafgyupqz6mmvxffdGd7XpaL2nBdVsTfhZ8rAdbjvcVlbc7H0xMMPDqwauqXl/wA/Q6000NjizRr2Da47yvNuNaEYWZdm3/wh9stb5XF7zUnwA3BakpOT2y8qqjVHpifBYnoEAQBAEAQAlSlshvXdkryEyOdbXdtMHNsrTjWQjYOHFWWNjerOa5flvZ/l1+fkXTBE1jQxgzWtFAAKAAahSisfBx0m5Pb8mdcT4eikxFcT4eiAVxPh6IBncQUDQzuIINDO4gg0eEg6CW0QkqT2hZE9iXWuyisR0yRt+XjaO7vGxaWTjKS2jqeI5Z79lb/b4kCa6ukKqaaemdbGSkto9WJIQBAEAQBAZxSuaQ5pIcNIIUp67kSipLT8Evuy8mWphikpn00g7cRituE1NaZSX488aXXDwZ2m0xWOPNbTOOobSe8VLca1pGMK7Muzql4IharS+Rxe81J6p9FqSbb2y7rrjXHpj4PisTMIAgCAIAgBKlLYbS7sleQmRzra4TTe7ZWn6GQjYOHFWWNjerOa5flvZr2dfn5F1QRtY0Mbmta0UAAAAA1BWJx0m5Pb8medxBDHQzuIINDO4gg0M7iCDR7pwUjsNOCDsNOCDsNOCDsCCdGhAVH7Q8h3Ql1sszaxHTJG0fBxtHd3jYtHJxupbR1PEcv3VVr/AG+JAGurpCqnHT0zrYyUltHqgyCAIAgCAIDKN5aQ4GhGkFF2IaTWmZTzue4vcSXHWSpbbe2RCEYLpiux81BkEAQBAEAQAlEtkNpLbJbkHkY+2uE8oLbK07dBkI2DhxVljY3qzmuX5ZVr2dfn5F1QQhjQxoa1rRQAaAANisTjpS6ntmenBSR2GnBB2GnBB2GnBB2GnBB2PM3AINjNwCDYzcAg2M3AINjNwCDYLK6M1vXJBsqT2h5DOhLrZZmViOmSMaS3e9o7uGxaOTjdXdHU8Ry+tVWv9viQBprpVW009M62MlJbQUGQQBAEAQBAEAQBAEAQBACUS2Q2l3ZLsg8i321wnlBbZWn6GQjYOHeVZY2N6s5rluXVa9nX5+RdMMDWNDGsaGgUAGoAbNSsNHHOTk9tmebgFJGxm4BBsZuAQbGbgEGxm4BBsZuAQbFMD1zUAUwPXNAKYHrmgFMD1zQCmB65oBTA9c0B4Wg6C005ftCdlSe0LIYwl1rsrCYjpkjHyb3tHdw2LSyMfq7o6jiOX1qq1/t8Svwa6VVtNPTOtjJSW0eqCQgCAIAgCAIAgCAIAUS2Q2l3ZLshMi321wnla5tlafoZCNg4d5VljY3qzm+W5ZVr2dfn5F1QwtY0MazNaBQAUAAGzWrDRx8pOT22Z0wPXNDEUwPXNAKYHrmgFMD1zQCmB65oBTA9c0ApgeuaAfl5qQPy80A/LzQD8vNAPy80A/LzQD8vNAeEA6DWnNCSoPaTka2zE2yCghcffjOjNcdra6wdy0cnHTW0dVw/KSb9lZ5IGCqtrR1ae1sISEAQBAEAQBAEAqgJVkBkl/OkMkhpZ4z7zQfeee7o1NxVhjY68s5zl+UdS6IeX97LvghaxoY0ZrQKAAUAA2KxOMk3J7Zn+XmpIH5eaAfl5oB+XmgH5eaAfl5oB+XmgH5eaAVxQCuKAVxQCuKAVxQCuKAVxQGne16RWaJ00smaxowqTuA2lYykkts9aaZ2zUILuURlXlPNb5c99WxNP/HHsA3ne5VeRkOT0jueN4yGPDb8/f8ABxarTLgIAgCAIAgCAIAgFUB0LhvuaxyiaF2n5mn4Xjcf2tmi9wZXZ+BXkw00Xxk3lBDbYRNE4jvMNM5rtrT+1bQmpLaOCycWePPpkvqdWuKzNcVxQCuKAVxQCuKAVxQCuKAVxQCuI65oBXEdc0AriOuaAVxHXNAK4jrmgFcR1zQGne16RWaJ00r2tY0czuAFdJWMpJLbPWmmVs1CC7lEZWZTzW+XPf7sTT/xxjUB3jvcVWZGQ5PSO443jIY8Nvz9/wAHEWmXIQBAEAQBAEAQBAEAQBAdG4L7msUwnhOn5mH4Xja0j/VsUXuDK3PwK8mDTRfGTeUEVthE0ThxNPxNO0HT5q2hNSW0cHk408efRJHVriOuazNcVxHXNAK4jrmgFcR1zQCuI65oBXEdc0AriOuaA9odwQCh3BAKHcEAodwQCh3BAaV73nFZonTSua1jfEnYBvKxlJJbZ6U0ytmoQ8lD5W5Ty2+XPd7sTf8Arj2Ad473HeqzIvcnpHc8bxsMeG359fv3HEWmXAQBAEAQBAEAQBAEAQBAEAQHRuC+5rFMJ4T9zT8L27Wkf7sWzRc4Mrs/AhkQaaL5yav+K2wiaKm5zSdLTuKtYTUltHB5ONPHn0TOtQ7gszXFDuCAUO4IBQ7ggFDuCAUO4IDynCEApwhAKcIQCnCEBpXveUNmidNMQ1jfppOxoG0rGTSW2etNU7ZqEPJRGVuU0tvlz3VbE3/rj2Ad473Heqy+9yekdxxvGxx4d/P3/Bw1qFwFACAIAgCAIAgCAIAgCAIAgCAKQdG4L7mscwniODmn4Xt2tI/3Ytim5wZXZ+DDIg0y+cm7+htsImipuc00zmncVawkpLaOEycaePPpmdanCFmawpwhAKcIQCnCEApwhAKYO8fVQBTB3j6oBTB3j6oDSve84bNE6eZxaxuOs7GgV0kqJSSW2etNM7ZqEPJQ+V2U0tvlz3VbE2vZx10Ad473Heqy+9yekdxxvGxx4bfn1+/ccNahcBAEAQBAEAQBAKqdEN67sVRpryQpJraCgyCAIAgCAIAgCA6WT99zWOYTxE7nNroe3ulbFNzgyuz8GGRBpovnJq/obbCJoidz210tduOlWsJqS2jhMnGnjz6Z/wDTrUwd4+qyNYUwd4+qAUwd4+qAUwd4+qAVxPh6KQK4nw9EBpXvekVmidPM/NY3ftOxo0aSdyxlJJbZ600ztmoQXcofK/KeW3y57qiJtezj2DiO9x3qsvvcnpHccbxsMeG35fk4S1S3CAIAgCAIAgCAIG9dyZ+z/It1scJ5g5tladxrIRsHDvK38fH9Wc5y3K+zXs6/PyOh7Q8hOxrarKw9l/8AJGAfc4hw79yzyMdNbRr8Tyzb9na+/v8AeV4Cq5prszqoyTW0FBIQBAEAQBAEAQHSyevyaxTCeI4Obse3ulbFNzgyvz8GGRBpovrJrKCG2wiaJxrqe35mO7p/atITUltHCZONPHn0zX1OtXE+HoszWFcT4eiAVxPh6IBncQQa+BpXve0NmidPLIGsaOZOxoG0ncsZSUVtnrTRO6ahBdyh8r8qJbfLnOq2JpPZs2DiO9yrb73J6R2/G8bDHht+TgrULgIAgCAIAgCAIBVNEN6JnkBkW62OE81W2Zp26DIRsHDvK38fH9Wc7yvK+zXs6/PyLtgjaxoY3Na1ooAKAADYt85CTcntmRIOgkU5KSCovaNkN2Rda7KKxnTJG3W3jaB8u8bFpZFCfdHUcTyr/wDFb/Z+8roFVzTXY6pSTW0FBIQBAEAQBAEAQHTydv2axTCaI4Obsc3ulbFNzgyuz8GGTDTL6yayhhtsImieK6nsNM5jtxH+7VaQmpLaOFycWePPpkvqdbO4gszW18BncQQa+B87VaGxsdI9zWsa0ucTsAFSVDejKEXKSjHyz8/5ZZUyW+XONWwtJ7Nm4d48RVZfe5PSO54zjY48Nvz6kfWqXAQBAEAQBAEAQCqBvRNcgMiH2xwnmBbZmnbrkI+UcO8rex8f1ZzvK8qql7Ov9XyLtgiDGhjQ1rQKAAUAA2LfOPlLqe2Z6cFJA04IDwiug0ogKi9o2QZiLrZZm1iOmSMfJve0d3DYtK+jfdHUcTy3iq19/R+8riqrmtdmdUmmtoISEAQBAEAQBAEB1MnL+msUwmiODm7Ht7p/a2KbnFldn4MMiDTP0Jct6MtUDLRGQWPFcQdRacQajkrSMupbRwV9MqbHCXlG7pwWR5EN9q9qMd3vAoM9zWGm4mpHkvG56gWfEQU8lfAotU7O/S0tBCQgCAIAgCAIBVNEN6JrkBkO+2OE8wLbM0/QyEfKOHeVvY+P6s57leWVS9nX+r5F2wwNY0MaxrWgUAGoAbNS39HHyk5PbZnTAdclJGxTAdckGxTAdckGxTAdckGwW10ZreuSDZUXtFyDMRdbLMysR0yRjTmna9o7uGxaV9G+6Oo4nlvFVr7+j95W9VXtaOqjJNbQUEhAEAQBAEAQBAW/7FbUXWeaLWGSAivGNPmFa4z3E4nnq1G5P3osamA65LYKLZF/aTdrp7BKGtJcykgG/M0keFV52x3FlhxlyryYtvz2/wAlA1VO1pn0GMtrYUEhAEAQBAEAUhvRNsgMh3WxwnmBFlB+hkI+UcO8rdox/VnPcryqqXs6/wBXyLthhaxoY1ma0CgAoAANi39HHyk5PbZnTA9c1JApgeuaAUwPXNAKYHrmgFMD1zQCmB65oDwtB0Fujl+1A2VF7RcgjEXWuysPZHTJGPlO1zQPlw2LTvo33R1HFct4qtff0fvK3qq9rR1Kaa2goJCAIAgCAIApDei6fY7dpjsbpnNP/M8lv2t90Hmaq1x46icNzdynkdK9Ce0wPXNe5THhA1afNAUt7RciDZnOtUDSbO41c2n/AFk/+P8AS0r6PVHXcTyvXquzz8/qQJaLWjpE0+6FVAFUAqgFUAQN6Jt7P8h3WxwnmDm2YHnIR8o4d5W7RRvuznuV5VVfl1/q+Rd0ELWNDGtzWtFABoAA1Bb5x8pOT2zP8vNCB+XmgH5eaAfl5oB+XmgH5eaAfl5oB+XmgPCAdGnzQFRe0XILss612Vp7LXJGB8G97eHDYtO+jfdHUcVyz7VWvv6P3lbVWg1o6lNNbQqoJFUAqgFUAQEsyFyOfbn57s5tmaffd3qfI3/TsW5RRvuyj5TlFRHpj+r77svezwMja2NgLWtADQK0AAoArBLRxcpOTbfk+n5eaGIrieuSkGMrGuBa7S0ihBAIIOsalBKbT2ilfaHkMbKTaYATZyfebtjJ/wDD+lo30eqOt4rlfafl2efn9SBrSaOjT33QQkIAmiG9E39n+RBtjhPNVtmB+hkI2Dh3lblFG+7Of5XlfZfl1/q+RdsETWNDGe61oAAAAAA1DUt/wchJuT2/J9K4nrkpMRXE9ckArieuSAVxPXJAK4nrkgFcT1yQCuJ65IBXE9ckArieuSAVxPXJAeGh0Emn0H6QFQ+0XIPss612VpMeuSMD4N7mgfLhsWlfRvujqOK5Z7VVvn0fvK2Wg1o6lS2toISEATQ2SzIXI19ufnuzmWZp9521x7jcd52Lboo33ZR8pyioXTHu/vuy9bDZI4Y2xRjNY0UDQBQDwVgkktI4yc5Tk5S7tn3rieuSkwFcT1yQGD5abR1zQGP8jEdc0BhLIHAtcGlpFCDpBB2KCU2ntFNZeZDOs5M9maXQE1LBpMfLWW/0tK7H9UdXxnL9X4LfPz+pAqrTcWjoVamKooth2JE3yDyIdanCadpZZwa0OgyYDaG4rbpx/VlDyfLKtdFfn5F1QOaxoYwMa1ooANAAGoa1veDkZNye35Pp/IxHXNSQP5GI65oB/IxHXNAP5GI65oB/IxHXNAP5GI65oB/IxHXNAP5GI65oB/IxHXNAP5GI65oB/IxHXNAP5GI65oDwz10HNp1igKk9oOQuYXWqytqw6XxN1t3uaN2C07qN90dPxfLvtXb/AJK2JWi4NHSxtTFUUWS7EiW5E5GyWxwkkBZZhrcdBdwt/a2qaN92UvJctGldMO8vvyXjYY44Y2xRtYxjRRrRqA8VvpaWkcfOcpycpd2z7/yMR1zUmBkyau0dc0BnnYjrmgMJYydOhAfPsXYIB2LsEA7F24IDh3lkTYpznSWWLOOtzasJ+ubSqwdcX6G3Vn31rUZvX+TG78hrDCQ5llizhqLyXkflVQq4r0Js5DIsWpTf9ux3hA7c1ehpjsXYIB2LsEA7F2CAdi7BAOxdggHYuwQDsXYIB2LsEA7F2CAdi7BAOxdggHYuwQDsXYIB2LsEA7F2CA4155HWO0HOls0RcfmFWu8W0qsHXF+UbVWbfUtRk9GvYsgrBEc5tljLhqLy5/k4kKFVFehnPkcia05v+3YkDbOQKANA3BehpbPexdggHYuwQGccRBqaID66cEBqrE9AgCAIAgCAIAgCAIAgCAIAgCAIAgCAIAgCAIAgCAIAgCAIAgP/2Q=="
    
    fig['layout']['images'] = [dict(
        source=img_source,
        x= Y,
        y= X,
        sizex= 0.15,
        sizey= 0.15,
        xanchor= "left",
        yanchor= "bottom"
      )]

    fig.append_trace({
        'z':V,
        'type': 'heatmap'
    }, 1, 1)
    fig.append_trace({
        'x':X_path,
        'y':Y_path,
        'type': 'scatter'
    }, 1, 1)

    return fig

viewer.show(app)